In [1]:
repo_url = 'https://github.com/fraimwork/fraimwork'

In [2]:
from utils.gitutils import clone_repo

# Clone the repo
repo = clone_repo(repo_url)
local_repo_path = str(repo.working_dir)

In [3]:
from utils.filetreeutils import FileTree

(file_tree := FileTree.from_dir(local_repo_path))

├── .git\
│   ├── hooks\
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── pre-receive.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── update.sample
│   ├── info\
│   │   ├── exclude
│   ├── logs\
│   │   ├── refs\
│   │   │   ├── heads\
│   │   │   │   ├── main
│   │   │   ├── remotes\
│   │   │   │   ├── origin\
│   │   │   │   │   ├── HEAD
│   │   ├── HEAD
│   ├── objects\
│   │   ├── 02\
│   │   │   ├── 878e0a3b87dc34cef3cca793115150d5288c39
│   │   ├── 03\
│   │   │   ├── adc8d20747d089c33376e807b93d7bf21923f1
│   │   ├── 08\
│   │   │   ├── 0d6c77ac21bb2ef88a6992b2b73ad93daaca92
│   │   ├── 0f\
│   │   │   ├── 8e2a9b7e8531d908ebeba6c72b7246a0c94d92
│   │   ├── 10\
│   │   │   ├── 26557942aa7ea18d8ef6a8e60078642fb3b1d2
│   │   │   ├── ceeb83899628dff4dbec6609

In [4]:
from utils.agent import Agent, GenerationConfig
from dotenv import load_dotenv
from utils.frameworkutils import Framework
import os

load_dotenv()

API_KEY = os.getenv('API_KEY')

frameworks = ', '.join(Framework.get_frameworks())

classifier = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="classifier",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a software engineer tasked with classifying the framework of a codebase into one of the  following frameworks:
{frameworks}.
-----------------------------------
In the following prompts, you will be given the subdirectories of a codebase. Directory names can be misleading. You can respond with one of the following:
- IDENTIFY: FRAMEWORK_NAME (Only select this if you are absolutely certain that the ROOT directory conforms) (it should be used sparingly)
- N/A (if you cannot identify the framework based on the subdirectories)
- END (if you are fairly certain that no frameworks are in the subdirectories)"""
)

In [5]:
import re
from utils.agent import Interaction
from utils.languageutils import get_imports
import networkx as nx
from utils.listutils import flatten, compact
# RegEx to match classifier responses
identify_re = re.compile(r"IDENTIFY: (.+)")
end_re = re.compile(r"END")
unable_re = re.compile(r"N/A")

def classify_node(node):
    node_dict = file_tree.nodes[node]
    if node_dict['name'] == '.git': return None
    neighbors = list(file_tree[node])
    prompt = f"{node_dict['name']}\n" + f"{file_tree.subfiletree(node).copy(withDepth=1)}"
    context = [Interaction(f'Relevant content of {neighbor}\n----\n' + '\n'.join(imports), '...') for  neighbor in  neighbors if 'content' in file_tree.nodes[neighbor] and (imports := get_imports(file_tree.nodes[neighbor]['path']))]
    response = classifier.chat(prompt, custom_context=context)
    # Match the response to the RegEx
    if (m := identify_re.match(response)):
        framework  = m.group(1).strip()
        print(f"{node_dict['name']} is a {framework} project")
        node_dict['frameworks'] = [framework]
        for node in nx.dfs_tree(file_tree, node).nodes:
            file_tree.nodes[node]['frameworks'] = [framework]
        return [framework]
    elif (m := unable_re.match(response)):
        print(f"Searching {node} ...")
        frameworks = compact(flatten([classify_node(node) for node in neighbors if 'content' not in file_tree.nodes[node]]))
        node_dict['frameworks'] = frameworks
    elif (m := end_re.match(response)):
        print(f"{node_dict['name']} is not a framework")
        return None
    else:
        print(f"Classifier response: {response}")
        return None

(root_classification := classify_node(file_tree.root_node()))

Searching . ...
.git is not a framework
.idx is not a framework
.vscode is not a framework
Searching backend ...
app is a FLASK project
firebase is a FIREBASE project
frontend is a REACT project
